In [1]:
from sqlalchemy import create_engine, Column, String, Integer, ForeignKey, Date, Text, DateTime, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

In [3]:
Base = declarative_base()

<ipython-input-3-9bae6e3862ba>:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
class Student(Base):
    __tablename__ = 'students'

    admission_no = Column(String, primary_key=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    course = Column(String, nullable=False)
    department = Column(String, nullable=False)
    year_of_study = Column(Integer, nullable=False)
    email = Column(String, unique=True, nullable=False)
    phone_number = Column(String, unique=True, nullable=False)
    gender = Column(Enum('Male', 'Female', 'Other'), nullable=False)
    date_of_birth = Column(Date, nullable=False)
    status = Column(String, nullable=False)

    internships = relationship('Internship', back_populates='student')
    evaluations = relationship('Evaluation', back_populates='student')
    feedbacks = relationship('Feedback', back_populates='student')

In [5]:
class Company(Base):
    __tablename__ = 'companies'

    company_id = Column(Integer, primary_key=True)
    company_name = Column(String, nullable=False)
    industry = Column(String, nullable=False)
    location_town = Column(String, nullable=False)
    street = Column(String, nullable=False)
    building = Column(String, nullable=False)
    telephone = Column(String, nullable=False)
    email = Column(String, nullable=False)
    website = Column(String)
    contact_person = Column(String)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    internships = relationship('Internship', back_populates='company')

In [6]:
class Supervisor(Base):
    __tablename__ = 'supervisors'

    supervisor_id = Column(Integer, primary_key=True)
    supervisor_name = Column(String, nullable=False)
    email = Column(String, unique=True, nullable=False)
    phone_number = Column(String, unique=True, nullable=False)
    department_id = Column(Integer, nullable=False)
    title = Column(String, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    internships = relationship('Internship', back_populates='supervisor')
    assignments = relationship('SupervisorAssignment', back_populates='supervisor')

In [7]:
class InternshipCategory(Base):
    __tablename__ = 'internship_categories'

    category_id = Column(Integer, primary_key=True)
    category_name = Column(String, nullable=False)
    description = Column(Text)

In [8]:
class Internship(Base):
    __tablename__ = 'internships'

    internship_id = Column(Integer, primary_key=True)
    admission_no = Column(String, ForeignKey('students.admission_no'), nullable=False)
    company_id = Column(Integer, ForeignKey('companies.company_id'), nullable=False)
    supervisor_id = Column(Integer, ForeignKey('supervisors.supervisor_id'), nullable=False)
    internship_start_date = Column(Date, nullable=False)
    internship_end_date = Column(Date, nullable=False)
    internship_location = Column(String, nullable=False)
    internship_description = Column(Text)
    status = Column(String, nullable=False)

    student = relationship('Student', back_populates='internships')
    company = relationship('Company', back_populates='internships')
    supervisor = relationship('Supervisor', back_populates='internships')
    evaluations = relationship('Evaluation', back_populates='internship')
    feedbacks = relationship('Feedback', back_populates='internship')

In [9]:
class Evaluation(Base):
    __tablename__ = 'evaluations'

    evaluation_id = Column(Integer, primary_key=True)
    internship_id = Column(Integer, ForeignKey('internships.internship_id'), nullable=False)
    student_id = Column(String, ForeignKey('students.admission_no'), nullable=False)
    evaluator_name = Column(String, nullable=False)
    evaluation_date = Column(Date, nullable=False)
    evaluation_score = Column(Integer, nullable=False)
    evaluation_comments = Column(Text)
    evaluation_type = Column(String, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    internship = relationship('Internship', back_populates='evaluations')
    student = relationship('Student', back_populates='evaluations')

In [10]:
class Feedback(Base):
    __tablename__ = 'feedbacks'

    feedback_id = Column(Integer, primary_key=True)
    internship_id = Column(Integer, ForeignKey('internships.internship_id'), nullable=False)
    student_id = Column(String, ForeignKey('students.admission_no'), nullable=False)
    student_feedback = Column(Text)
    supervisor_feedback = Column(Text)
    feedback_type = Column(String, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    internship = relationship('Internship', back_populates='feedbacks')
    student = relationship('Student', back_populates='feedbacks')

In [11]:
class SupervisorAssignment(Base):
    __tablename__ = 'supervisors_assignment'

    assignment_id = Column(Integer, primary_key=True)
    supervisor_id = Column(Integer, ForeignKey('supervisors.supervisor_id'), nullable=False)
    supervisor_order = Column(Integer, nullable=False)
    date_range = Column(String, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

    supervisor = relationship('Supervisor', back_populates='assignments')

In [12]:
def create_db():
    engine = create_engine('sqlite:///industrial_attachment.db')
    Base.metadata.create_all(engine)

    # Create a new session
    Session = sessionmaker(bind=engine)
    session = Session()

    # Example of inserting data
    # student = Student(
    #     admission_no='1037980',
    #     first_name='James',
    #     last_name='Sifuna',
    #     course='Bachelor of Science in Physics',
    #     department='Natural Science',
    #     year_of_study=3,
    #     email='sifunajames@gmail.com',
    #     phone_number='0725499073',
    #     gender='Male',
    #     date_of_birth='1989-09-25',
    #     status='Active'
    # )
    # session.add(student)
    # session.commit()

    session.close()
